Iterate over images saved by the webcam and capture all predictions

In [21]:
img_dir = "/Users/robin/Pictures/"

In [22]:
import glob
from PIL import Image, ImageDraw
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

IP = "localhost"
PORT = "5000"

TIME_FORMAT = "%Y-%m-%d_%H:%M:%S"
TIMEOUT = 10 # seconds for a request

TARGET = 'person'

In [23]:
files = glob.glob(img_dir + "*.jpg")

In [24]:
files[:5]

['/Users/robin/Pictures/filezilla_hassio.jpg',
 '/Users/robin/Pictures/blank.jpg',
 '/Users/robin/Pictures/family.jpg',
 '/Users/robin/Pictures/medicine.jpg',
 '/Users/robin/Pictures/black.jpg']

In [25]:
for file in files[:5]:
    print(file)

/Users/robin/Pictures/filezilla_hassio.jpg
/Users/robin/Pictures/blank.jpg
/Users/robin/Pictures/family.jpg
/Users/robin/Pictures/medicine.jpg
/Users/robin/Pictures/black.jpg


Process the image

In [27]:
def process_image(file_path: str) -> dict:
    """
    Process an image with deepstack.
    """
    try:
        image_bytes = open(file_path, "rb").read()
        print(f"opened {file_path}")
        response = requests.post(f"http://{IP}:{PORT}/v1/vision/detection", files={"image":image_bytes}, timeout=TIMEOUT)
        predictions = response.json()['predictions']
        print(f"Total of {len(predictions)} predictions")
        print("--------")
        return predictions
    except Exception as exc:
        print(exc)

In [28]:
predictions = process_image(files[2])

opened /Users/robin/Pictures/family.jpg
Total of 2 predictions
--------


We can now append each prediction as a row in a table

In [29]:
predictions_list = []

for file_path in files:
    predictions = None
    predictions = process_image(file_path)
    if predictions:
        for prediction in predictions:
            prediction["file_path"] = file_path
            predictions_list.append(prediction)

opened /Users/robin/Pictures/filezilla_hassio.jpg
Total of 0 predictions
--------
opened /Users/robin/Pictures/blank.jpg
Total of 0 predictions
--------
opened /Users/robin/Pictures/family.jpg
Total of 2 predictions
--------
opened /Users/robin/Pictures/medicine.jpg
Total of 0 predictions
--------
opened /Users/robin/Pictures/black.jpg
Total of 0 predictions
--------
opened /Users/robin/Pictures/test-image3.jpg
Total of 3 predictions
--------
opened /Users/robin/Pictures/fishtank.jpg
Total of 0 predictions
--------


In [30]:
df = pd.DataFrame(predictions_list)

In [31]:
df["confidence"] = pd.to_numeric(df["confidence"])

In [32]:
df.head()

,confidence,file_path,label,x_max,x_min,y_max,y_min
0,0.998541,/Users/robin/Pictures/family.jpg,person,768,327,627,53
1,0.994190,/Users/robin/Pictures/family.jpg,person,1015,556,647,0
2,0.999866,/Users/robin/Pictures/test-image3.jpg,person,485,258,676,0
3,0.999655,/Users/robin/Pictures/test-image3.jpg,person,639,405,652,0
4,0.997456,/Users/robin/Pictures/test-image3.jpg,dog,825,624,591,311


In [33]:
df.to_csv('image_record.csv')

In [34]:
persons_df = df[df["label"] == 'person'].groupby(["file_path"]).count()["confidence"]

In [35]:
persons_df = persons_df.reset_index()

In [36]:
persons_df.head()

,file_path,confidence
0,/Users/robin/Pictures/family.jpg,2
1,/Users/robin/Pictures/test-image3.jpg,2


## Do some queries
See what objects are given a low score

In [38]:
all_counts = df.groupby(['label']).count()["confidence"]
all_counts

label
dog       1
person    4
Name: confidence, dtype: int64